# Output Parser
**Output Parser**는 대규모 언어 모델(LLM, Large Language Model)의 출력 결과를 **애플리케이션에서 활용할 수 있도록 적절한 형식으로 변환**하는 도구이다.
- LLM은 일반적으로 텍스트 형태로 응답을 생성하지만, 이 텍스트는 그대로 활용하기 어려운 경우가 많다.
- Output Parser는 이러한 **비구조적 텍스트 데이터를 구조화된 데이터로 변환**하여 프로그램에서 활용 가능하도록 만든다.
- 예를 들어, 키워드 리스트를 뽑거나 JSON 형식으로 정보를 변환하는 데 사용된다.

## 주요 Output Parser 종류

1. **CommaSeparatedListOutputParser**
   - 쉼표로 구분된 텍스트를 파싱하여 리스트 형태로 변환한다.
   - 쉼표로 구분된 텍스트를 응답 받을 수 있게 프롬포트를 작성해야함.
   - 예: `"사과, 바나나, 포도"` → `["사과", "바나나", "포도"]`
2. **JsonOutputParser**
   - LLM의 출력이 JSON 형식일 때 이를 Python의 `dict` 객체로 변환한다.
   - JSON(JavaScript Object Notation)은 데이터 구조를 표현하기 위한 경량 포맷이다.
   - `dict[key]`로 조회 가능
3. **PydanticOutputParser**
   - JSON 데이터를 Python의 [Pydantic](https://docs.pydantic.dev) 모델로 변환한다.
   - Pydantic은 데이터 유효성 검사와 설정 관리에 널리 사용되는 Python 라이브러리이다.
   - `.[key]`로 조회 가능
4. **StrOutputParser**
   - 모델의 출력 결과를 단순 문자열로 반환한다.
   - Chat 기반 모델은 Message 객체의 속성으로 LLM 결과를 반환한다. 거기에서 응답 문자열만 추출해서 반환한다.
> `JsonOutputParser`, `PydanticOutputParser` 는 모두 Pydantic을 사용해 데이터 구조(schema)를 정의하고, 해당 구조에 따라 출력을 검증하고 변환한다.

## 주요 메소드
- `parse(text: str)`
  - LLM이 생성한 문자열 응답을 받아 정해진 구조로 변환하여 반환한다.
- `get_format_instructions() -> str`
  - 각 OutputParer가 변환할 수있는 형식으로 LLM이 응답하도록 하는 프롬프트 텍스트를 반환한다.
  - 이 내용을 프롬프트에 넣어서 LLM이 정확한 포맷으로 응답하도록 유도한다.
  
## 참고
- Output Parser는 일반적으로 [`Runnable`](05_chaing_LECL.ipynb#Runnable) 인터페이스를 상속하여 구현되며, `invoke()` 메서드를 통해 실행할 수 있다.
- `invoke()`는 내부적으로 `parse()`를 호출하여 동작한다.
- 필요한 경우 Output Parser를 직접 구현하여 사용자 정의 출력 포맷을 처리할 수도 있다. 


In [2]:
from dotenv import load_dotenv

load_dotenv()

True

## StrOutputParser
- 모델(LLM)의 출력 결과를 string으로 변환하여 반환하는 output parser.
- Chat Model은  Message 객체에서 content 속성값을 추출하여 문자열로 반환한다.

In [134]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# prompt 생성
prompt_template = ChatPromptTemplate.from_template(
    "한국의 {topic} 관련된 속담을 {count}개 알려줘"
)
prompt = prompt_template.format(topic = "호랑이", count = 2)

# LLM 모델 생성
model = ChatOpenAI(model_name = "gpt-4o-mini")

# LLM 모델에 prompt를 전달하고 응답 받기
## prompt - > llm model -> response
response = model.invoke(prompt)

In [140]:
print(response)

content='한국의 호랑이에 관련된 속담 중 두 가지는 다음과 같습니다:\n\n1. **호랑이 굴에 가야 호랑이 새끼를 잡는다** - 이 속담은 목표를 이루기 위해서는 위험을 감수해야 한다는 뜻입니다. 원하는 것을 얻기 위해서는 적극적으로 행동해야 한다는 의미를 담고 있습니다.\n\n2. **호랑이도 제 말 하면 온다** - 이 속담은 누군가에 대해 이야기하고 있을 때 그 사람이 갑자기 나타나는 상황을 비유적으로 표현한 것입니다. 말이란 것을 통해 상대방이 나타날 수 있음을 의미합니다.\n\n이 두 속담은 호랑이를 통해 위험, 행동, 그리고 우연의 연관성을 잘 보여줍니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 161, 'prompt_tokens': 24, 'total_tokens': 185, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-BgTTZ0VZ0OsmtXA5cRXLHO3AHAzl8', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--d225efb9-c936-4b57-abc0-cfc7cbf659a9-0' usage_metadata={'input_tokens': 24, 'output_tokens': 161, 'total_tokens': 185, 'in

In [138]:
res1 = response.content
print(res1)

한국의 호랑이에 관련된 속담 중 두 가지는 다음과 같습니다:

1. **호랑이 굴에 가야 호랑이 새끼를 잡는다** - 이 속담은 목표를 이루기 위해서는 위험을 감수해야 한다는 뜻입니다. 원하는 것을 얻기 위해서는 적극적으로 행동해야 한다는 의미를 담고 있습니다.

2. **호랑이도 제 말 하면 온다** - 이 속담은 누군가에 대해 이야기하고 있을 때 그 사람이 갑자기 나타나는 상황을 비유적으로 표현한 것입니다. 말이란 것을 통해 상대방이 나타날 수 있음을 의미합니다.

이 두 속담은 호랑이를 통해 위험, 행동, 그리고 우연의 연관성을 잘 보여줍니다.


In [141]:
parser = StrOutputParser()		# Message 객체에서 content 속성(메세지)의 값만 추출.
res = parser.invoke(response)	# LLM 모델의 응답 결과, 걍 .content하면 되지 이렇게 왜함 ?? -> chain쪽 들어가면 편해진대용
print(res)

한국의 호랑이에 관련된 속담 중 두 가지는 다음과 같습니다:

1. **호랑이 굴에 가야 호랑이 새끼를 잡는다** - 이 속담은 목표를 이루기 위해서는 위험을 감수해야 한다는 뜻입니다. 원하는 것을 얻기 위해서는 적극적으로 행동해야 한다는 의미를 담고 있습니다.

2. **호랑이도 제 말 하면 온다** - 이 속담은 누군가에 대해 이야기하고 있을 때 그 사람이 갑자기 나타나는 상황을 비유적으로 표현한 것입니다. 말이란 것을 통해 상대방이 나타날 수 있음을 의미합니다.

이 두 속담은 호랑이를 통해 위험, 행동, 그리고 우연의 연관성을 잘 보여줍니다.


In [ ]:
# prompt_template -> model -> output parser
chain = prompt_template | model | parser		# |(or) 연산자로 묶어줌, pipeline으로 생각하면 좋음.

res = chain.invoke({"topic" : "사람의 정신력", "count" : 3})
print(res)

한국의 정신력과 관련된 속담은 다음과 같습니다:

1. **"가는 말이 고와야 오는 말이 곱다."**  
   이 속담은 자신의 행동과 말이 타인에게 어떻게 영향을 미치는지를 강조합니다. 긍정적인 태도로 임하면 상대방도 긍정적으로 반응한다는 의미로, 정신적 힘과 긍정적인 마인드를 강조합니다.

2. **"하늘이 무너져도 솟아날 구멍은 있다."**  
   절망적인 상황에서도 희망과 해결책이 있다는 의미로, 어려운 상황에서도 포기하지 않고 극복할 수 있는 정신적인 힘을 상징합니다.

3. **"백문이 불여일견."**  
   많은 말을 듣는 것보다 직접 경험하는 것이 더 중요하다는 뜻으로, 자신의 눈으로 보고 느끼는 것이 정신력의 중요성을 나타냅니다. 스스로 경험하고 도전하는 정신력이 강조됩니다.

이 속담들은 어려움을 극복하고 긍정적인 마음가짐을 유지하는데 중요한 메시지를 담고 있습니다.


In [ ]:
# prompt_template -> model -> output parser
chain = prompt_template | model #| parser		# 이러면 model의 출력값을 출력하게 됨.

res = chain.invoke({"topic" : "사람의 정신력", "count" : 3})
print(res)

content='한국의 정신력과 관련된 속담은 다음과 같습니다:\n\n1. **"뛰는 놈 위에 나는 놈 있다"**  \n   - 아무리 뛰어난 사람도 그보다 더 뛰어난 사람이 있다는 뜻으로, 사람의 노력과 정신력이 중요한 동시에, 자신의 한계를 인식하는 것이 필요하다는 메시지를 담고 있습니다.\n\n2. **"꾸준한 바위에 물이 스며든다"**  \n   - 꾸준한 노력과 인내가 결국 목표를 이룰 수 있다는 의미로, 정신력과 끈기를 강조하는 속담입니다.\n\n3. **"고생 끝에 낙이 온다"**  \n   - 어려운 시기를 견디면 결국 좋은 결과가 올 것이라는 뜻으로, 힘든 상황에서도 정신적으로 버텨야 한다는 교훈을 줍니다.\n\n이 속담들은 한국 문화에서의 인내, 노력, 정신력의 중요성을 잘 나타내고 있습니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 205, 'prompt_tokens': 23, 'total_tokens': 228, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-BgLg2GvGIBM9cqop1NA72Adp8yKjh', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--78aa239f-3750-4afc-a661-f945aad1fa0c-0' usage

In [143]:
print(res)
parser = CommaSeparatedListOutputParser()
parser.invoke(res)

한국의 호랑이에 관련된 속담 중 두 가지는 다음과 같습니다:

1. **호랑이 굴에 가야 호랑이 새끼를 잡는다** - 이 속담은 목표를 이루기 위해서는 위험을 감수해야 한다는 뜻입니다. 원하는 것을 얻기 위해서는 적극적으로 행동해야 한다는 의미를 담고 있습니다.

2. **호랑이도 제 말 하면 온다** - 이 속담은 누군가에 대해 이야기하고 있을 때 그 사람이 갑자기 나타나는 상황을 비유적으로 표현한 것입니다. 말이란 것을 통해 상대방이 나타날 수 있음을 의미합니다.

이 두 속담은 호랑이를 통해 위험, 행동, 그리고 우연의 연관성을 잘 보여줍니다.


['한국의 호랑이에 관련된 속담 중 두 가지는 다음과 같습니다:',
 '1. **호랑이 굴에 가야 호랑이 새끼를 잡는다** - 이 속담은 목표를 이루기 위해서는 위험을 감수해야 한다는 뜻입니다. 원하는 것을 얻기 위해서는 적극적으로 행동해야 한다는 의미를 담고 있습니다.',
 '2. **호랑이도 제 말 하면 온다** - 이 속담은 누군가에 대해 이야기하고 있을 때 그 사람이 갑자기 나타나는 상황을 비유적으로 표현한 것입니다. 말이란 것을 통해 상대방이 나타날 수 있음을 의미합니다.',
 '이 두 속담은 호랑이를 통해 위험',
 '행동',
 '그리고 우연의 연관성을 잘 보여줍니다.']

## CommaSeparatedListOutputParser

- 쉼표로 구분된 텍스트를 파싱하여 리스트 형태로 변환한다.
  - "a,b,c" => ['a','b','c']

In [28]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

parser = CommaSeparatedListOutputParser()
res_txt = "사과, 배, 귤, 수박, 오렌지"
print(parser.invoke(res_txt))

['사과', '배', '귤', '수박', '오렌지']


In [ ]:
# 출력 형식을 지정하는 프롬프트를 조회
format_string = parser.get_format_instructions()
print(format_string)

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [55]:
from textwrap import dedent	# 공백처리하는 친구, 보기좋게
prompt_template = ChatPromptTemplate.from_template(
    dedent("""
	# instruction
	{subject}의 이름 다섯 개를 나열해주세요.
           
    # output indicator (출력 형식)
	{format_instructions}
	"""),
    partial_variables={"format_instructions" : parser.get_format_instructions()}	# 이 메소드의 리턴 값을 파라미터에 넣어서 템플릿 안에 넣은 것.
) # 실제로 이 template의 placeholder는 {subject}뿐 !
# partial_variables={변수명:넣을 값, ...} : template의 placeholder 변수에 넣을 값을 PromptTemplate객체를 생성할 때 넣겠다. 
# placeholder에 넣을 값이 있는데 함수나 메소드 호출을 통한 리턴 값을 가져와야하는 경우에 사용.

model = ChatOpenAI(model_name = "gpt-4o-mini")

# prompt 생성
prompt = prompt_template.invoke({"subject" : " 동물"})

# LLM에 요청
response = model.invoke(prompt)

In [56]:
# 응답 확인
response.content

# parser를 이용해서 list로 변환
res = parser.invoke(response)

print(response.content)
print(type(res), res, response.content.split(","))	# split으로도 가능.

호랑이, 사자, 기린, 코끼리, 펭귄
<class 'list'> ['호랑이', '사자', '기린', '코끼리', '펭귄'] ['호랑이', ' 사자', ' 기린', ' 코끼리', ' 펭귄']


In [57]:
# chain으로 구성
chain = prompt_template | model | parser

res = chain.invoke({"subject" : "게임"})
res

['롤', '오버워치', '마인크래프트', '배틀그라운드', '포트나이트']

## JsonOutputParser

- JSON 형식의 응답을 dictionary로 반환한다.
- JSON 형식을 정하려는 경우 [Pydantic](Ref_typing_Pydantic.ipynb)을 이용해 JSON 스키마를 정의하여 JsonOutputParser 생성시 전달한다.
  - Pydantic 모델클래스를 이용해 LLM 모델이 응답할 때 json의 어떤 key에 어떤 응답을 작성할 지 Field로 정의한다.
  - Schema 지정은 필수는 아니다. 
- LLM이 JSON Schema를 따르는 형태로 응답을 하면 JsonOutputParser는 Dictionary로 변환한다.

In [146]:
from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser()
res_text = """
{
	"name":"홍길동",
    "age":20,
    "address":"대구",
    "hobby":["독서", "게임"]
}
"""

res_dict = parser.invoke(res_text)
print(type(res_text), type(res_dict))
res_dict, res_dict["name"], res_dict["address"]

<class 'str'> <class 'dict'>


({'name': '홍길동', 'age': 20, 'address': '대구', 'hobby': ['독서', '게임']},
 '홍길동',
 '대구')

In [63]:
parser.get_format_instructions()

'Return a JSON object.'

In [76]:
prompt_template = ChatPromptTemplate.from_template(
    "{name}에 대해서 설명해줭. \n{format_instructions}",
    partial_variables = {"format_instructions" : parser.get_format_instructions()}	# json으로 출력하라는 내용이 prompt에 들어가니 json 형식으로 출력
)
model = ChatOpenAI(model_name = "gpt-4o-mini")

prompt = prompt_template.invoke({"name" : "아이폰"})

res = model.invoke(prompt)

In [79]:
print(res.content)	 # json 형태로 출력된 것을 확인 !

Sure! Here is a JSON object that provides a brief overview of the iPhone:

```json
{
  "smartphone": {
    "name": "iPhone",
    "manufacturer": "Apple Inc.",
    "first_release": "June 29, 2007",
    "latest_model": {
      "name": "iPhone 14",
      "release_date": "September 16, 2022",
      "features": [
        "6.1-inch Super Retina XDR display",
        "A15 Bionic chip",
        "Dual 12MP camera system",
        "Face ID technology",
        "5G capable",
        "iOS 16"
      ]
    },
    "operating_system": "iOS",
    "key_features": [
      "High-quality camera",
      "App Store with a wide range of applications",
      "Integration with other Apple products",
      "Regular software updates",
      "Sleek design and build quality"
    ],
    "target_audience": {
      "demographics": "General consumers, professionals, tech enthusiasts",
      "market_position": "Premium smartphone segment"
    }
  }
}
```

This JSON object outlines key information about the iPhone, inclu

In [80]:
prompt_template = ChatPromptTemplate.from_template(
    "{name}에 대해서 설명해줘, json형태로" # 그냥 자연어로 json 형태로 출력해달라고 말해도 됨
    # partial_variables = {"format_instructions" : parser.get_format_instructions()}	# json으로 출력하라는 내용이 prompt에 들어가니 json 형식으로 출력
)
model = ChatOpenAI(model_name = "gpt-4o-mini")

prompt = prompt_template.invoke({"name" : "아이폰"})

res = model.invoke(prompt)

In [81]:
print(res.content)

아래는 아이폰에 대한 정보를 JSON 형식으로 나타낸 것입니다.

```json
{
  "제품명": "아이폰",
  "제조사": "애플",
  "출시연도": 2007,
  "현재모델": {
    "모델명": "아이폰 15",
    "출시연도": 2023,
    "특징": {
      "디스플레이": "Super Retina XDR",
      "카메라": {
        "후면": {
          "수": 3,
          "해상도": "48MP"
        },
        "전면": {
          "해상도": "12MP"
        }
      },
      "프로세서": "A17 Bionic",
      "배터리": "최대 20시간 통화",
      "운영체제": "iOS 17"
    }
  },
  "주요기능": [
    "전화 및 메시지 전송",
    "인터넷 브라우징",
    "앱 생태계",
    "카메라 촬영",
    "음악 및 비디오 스트리밍",
    "Face ID 및 Touch ID 보안",
    "아이메시지 및 페이스타임"
  ],
  "지원하는기술": [
    "5G",
    "Wi-Fi 6E",
    "블루투스 5.3",
    "U1 칩 (초광대역 위치 인식)"
  ]
}
```

위 JSON 데이터는 아이폰의 기본 정보, 현재 모델 특징, 주요 기능, 그리고 지원하는 기술들에 대해 설명하고 있습니다.


In [84]:
res_dict = parser.invoke(res)
print(type(res.content), type(res_dict))

<class 'str'> <class 'dict'>


In [89]:
res_dict["현재모델"]["출시연도"]

2023

In [90]:
print(parser.get_format_instructions())	

Return a JSON object.


In [107]:
# 출력 schema(설계도)를 정의
## json 형식을 설계
from platform import release
from pydantic import BaseModel, Field

class ItemSchema(BaseModel):
    # JSON에 포함될 항목들을 class변수로 정의. 
	# 변수명 : type = Field(설명)
    name : str = Field(description="제품의 이름")
    info : str = Field(description="제품에 대한 정보")
    release_date : str = Field(description="제품 출시 일시 yyyy-mm-dd 형식")
    price : int = Field(description="제품의 한국 가격")
    
parser = JsonOutputParser(pydantic_object=ItemSchema)
print(parser.get_format_instructions())		# 설명이 추가된 모습.

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"description": "제품의 이름", "title": "Name", "type": "string"}, "info": {"description": "제품에 대한 정보", "title": "Info", "type": "string"}, "release_date": {"description": "제품 출시 일시 yyyy-mm-dd 형식", "title": "Release Date", "type": "string"}, "price": {"description": "제품의 한국 가격", "title": "Price", "type": "integer"}}, "required": ["name", "info", "release_date", "price"]}
```


In [108]:
prompt_template = ChatPromptTemplate.from_template(
    dedent("""
	# instruction
	{name}에 대해서 설명해주세요.
           
	# output indicator
	{format_instructions}
	"""),
    partial_variables = {"format_instructions" : parser.get_format_instructions()}
)

model = ChatOpenAI(model_name = "gpt-4o-mini")

In [112]:
p = prompt_template.invoke({"name" : "갤럭시폰"})
print(p.messages[0].content)


# instruction
갤럭시폰에 대해서 설명해주세요.

# output indicator
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"description": "제품의 이름", "title": "Name", "type": "string"}, "info": {"description": "제품에 대한 정보", "title": "Info", "type": "string"}, "release_date": {"description": "제품 출시 일시 yyyy-mm-dd 형식", "title": "Release Date", "type": "string"}, "price": {"description": "제품의 한국 가격", "title": "Price", "type": "integer"}}, "required": ["name", "info", "release_date", "price"]}
```



In [149]:
res = model.invoke(p)
print(type(res.content), res.content)
response = parser.invoke(res)
type(response), response

<class 'str'> ```json
{
  "name": "갤럭시 S23",
  "info": "갤럭시 S23는 삼성전자가 출시한 최신 스마트폰으로, 뛰어난 카메라 성능과 고속 프로세서를 특징으로 합니다. 또한, AMOLED 디스플레이와 5G 연결을 지원하여 뛰어난 사용자 경험을 제공합니다.",
  "release_date": "2023-02-17",
  "price": 1200000
}
```


(dict,
 {'name': '갤럭시 S23',
  'info': '갤럭시 S23는 삼성전자가 출시한 최신 스마트폰으로, 뛰어난 카메라 성능과 고속 프로세서를 특징으로 합니다. 또한, AMOLED 디스플레이와 5G 연결을 지원하여 뛰어난 사용자 경험을 제공합니다.',
  'release_date': '2023-02-17',
  'price': 1200000})

## PydanticOutputParser

- JSON 형태로 받은 응답을 Pydantic 모델로 변환하여 반환한다.
- 구현은 JsonOutputParser와 동일한데 parsing 결과를 pydantic 모델타입으로 반환한다.

In [ ]:
parser = PydanticOutputParser()
# 얘는 반드시 pydantic_object를 줘야함, Pydantic model로 parsing해야 하기 때문에 
# 어떤 기준의 Pydantic model class로 parsing 할지 알려줘야함.

ValidationError: 1 validation error for PydanticOutputParser
pydantic_object
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing

In [154]:
from langchain_core.output_parsers import PydanticOutputParser

parser = PydanticOutputParser(pydantic_object=ItemSchema)
# JsonOUtputParser와 동일한 format instruction을 생성
## 응답 : JSON -> Parser => Pydantic Model객체로 변환.
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"description": "제품의 이름", "title": "Name", "type": "string"}, "info": {"description": "제품에 대한 정보", "title": "Info", "type": "string"}, "release_date": {"description": "제품 출시 일시 yyyy-mm-dd 형식", "title": "Release Date", "type": "string"}, "price": {"description": "제품의 한국 가격", "title": "Price", "type": "integer"}}, "required": ["name", "info", "release_date", "price"]}
```


In [125]:
prompt_template = ChatPromptTemplate.from_template(
    dedent("""
	# instruction
	{name}에 대해서 설명해주세요.
           
	# output indicator
	{format_instructions}
	"""),
    partial_variables = {"format_instructions" : parser.get_format_instructions()}
)

model = ChatOpenAI(model_name = "gpt-4o-mini")

In [126]:
prompt = prompt_template.invoke({"name" : "Mac BOok"})
res = model.invoke(prompt)
response = parser.invoke(res)

In [127]:
response

ItemSchema(name='MacBook', info='MacBook은 Apple에서 제조한 노트북 컴퓨터로, 고급스러운 디자인과 안정적인 성능을 자랑합니다. macOS 운영 체제를 사용하며, 다양한 멀티미디어 작업에 최적화되어 있습니다.', release_date='2023-01-24', price=1999000)

In [128]:
type(response)

__main__.ItemSchema

In [129]:
print("제품 이름 : ", response.name)
print("정보 : ", response.info)
print(response.release_date, response.price)

제품 이름 :  MacBook
정보 :  MacBook은 Apple에서 제조한 노트북 컴퓨터로, 고급스러운 디자인과 안정적인 성능을 자랑합니다. macOS 운영 체제를 사용하며, 다양한 멀티미디어 작업에 최적화되어 있습니다.
2023-01-24 1999000


In [131]:
chain = prompt_template | model | parser

response = chain.invoke({"name" : "아이폰"})
response

ItemSchema(name='아이폰', info='아이폰은 애플(Apple)에서 개발한 스마트폰으로, iOS 운영 체제를 사용하며 다양한 애플리케이션을 지원합니다. 스타일리시한 디자인과 뛰어난 카메라 성능, 사용자 친화적인 인터페이스로 유명합니다.', release_date='2023-09-22', price=1350000)